In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
con=sqlite3.connect("database.sqlite")

In [11]:
filtered_data=pd.read_sql_query("""SELECT * FROM REVIEWS WHERE Score !=3 LIMIT 5000""",con)
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
4995,5423,B00622CYVS,A17ASMX6QMO6XY,E. Harvill,0,1,2,1277424000,Not so tasty...,"My baby didn't seem into these dinners, so I t..."
4996,5424,B00622CYVS,A32DHN8U74GCAR,"Granola Girl ""michele j.""",0,1,4,1240790400,Food Delivery,This is great! Organic baby food options - de...
4997,5425,B00622CYVS,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0,1,5,1236988800,Dinner time is Earths Best TIme !!,My little guy loves to try new foods..so this ...
4998,5426,B00622CYVS,A2NYT3UXUTBY23,C&GHoll,1,3,2,1249603200,Wrong item shipped,We ordered the Earth's best 2nd dinner variety...


In [12]:
def partition(x):
    if x<3:
        return "Negative"
    return "Positive"

In [13]:
actualScore=filtered_data["Score"]
posneg=actualScore.map(partition)
filtered_data["Score"]=posneg
filtered_data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [15]:
display.shape

(80668, 7)

In [16]:
display.head(5)

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [17]:
display[display["UserId"]=='AZY10LLTJ71NX']

,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
80638,AZY10LLTJ71NX,B001ATMQK2,"undertheshrine ""undertheshrine""",1296691200,5,I bought this 6 pack because for the price tha...,5


In [18]:
display["COUNT(*)"].sum()

393063

In [20]:
display=pd.read_sql_query("""SELECT * FROM REVIEWS WHERE SCORE !=3 AND UserId="AR5J8UI46CURR" ORDER BY ProductId""",con)

In [21]:
display.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [22]:
sorted_data=filtered_data.sort_values("ProductId",axis=0,
    ascending=True,
    inplace=False,
    kind='quicksort',
    na_position='last')

In [23]:
final=sorted_data.drop_duplicates(subset={"UserId","Score","Time","Text"},keep='first',inplace=False)
final.shape

(4986, 10)

In [25]:
(final["Id"].size/filtered_data["Id"].size)*100

99.72

In [26]:
duplicate=pd.read_sql_query("""SELECT * FROM REVIEWS WHERE HelpfulnessDenominator<HelpfulnessNumerator""",con)

In [27]:
duplicate

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [28]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [29]:
final.shape

(4986, 10)

In [34]:
final["Score"].value_counts()

Positive    4178
Negative     808
Name: Score, dtype: int64

In [40]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,Positive,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,Positive,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,Positive,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,Positive,961718400,Great Product,This was a really good idea and the final prod...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,Positive,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...


In [41]:
count_vector=CountVectorizer(stop_words='english')
count_vector.fit(final["Text"])

CountVectorizer(stop_words='english')

In [45]:
print("Print some features",count_vector.get_feature_names())

Print some features ['00', '000', '000kwh', '002', '0100', '0174', '02', '03', '03510', '04', '042608460503', '0472066978', '06', '0738551856', '09', '090', '0g', '0mg', '0xk6hzpjrkaed855hewp', '0z', '10', '100', '1000', '100ml', '100s', '100x', '101', '103', '108', '10g', '10lb', '10lbs', '10mg', '10oz', '10th', '10x', '11', '110', '110mg', '114', '11g', '12', '120', '121', '1245', '1246', '125', '129', '12g', '12lbs', '12oz', '13', '130', '1300131716', '1300watt', '1306', '134', '135', '135mg', '1388', '13oz', '14', '140', '144', '15', '150', '150mg', '15g', '15lb', '15lbs', '15mg', '15oz', '16', '160', '160lb', '160mg', '167', '16g', '16lb', '16oz', '17', '170', '170g', '17g', '17oz', '18', '180', '180mg', '184', '1845', '1845979125', '1866', '1898', '18g', '18oz', '18th', '19', '190', '1912', '1933981350', '1952', '1980', '1980s', '1987', '1990s', '1996', '1998', '19ish', '19oz', '19th', '1c', '1g', '1lb', '1lbs', '1oz', '1st', '1t', '1x850g', '20', '200', '2000', '2003', '2007', '

In [46]:
final_counts=count_vector.transform(final["Text"])

In [48]:
final_counts.shape

(4986, 13217)

In [65]:
bi_gram=CountVectorizer(ngram_range=(1,5))
bigram_counts=bi_gram.fit_transform(final["Text"])

In [66]:
bigram_counts.shape

(4986, 1057604)

In [67]:
bigram_counts.get_shape()

(4986, 1057604)

In [72]:
tfid=TfidfVectorizer(ngram_range=(1,2),min_df=10)
tfid_counts=tfid.fit_transform(final["Text"])

In [73]:
tfid_counts.get_shape()

(4986, 6840)

In [78]:
print(tfid.get_feature_names())

['00', '10', '10 years', '100', '100 calorie', '100 calories', '11', '12', '12 oz', '13', '14', '15', '15 minutes', '16', '16 ounce', '16 oz', '17', '18', '1g', '1st', '20', '200', '2010', '2012', '24', '24 pack', '25', '2g', '2nd', '30', '36', '3rd', '40', '45', '50', '60', '60 calories', '70', '75', '80', '8oz', '90', '99', 'ability', 'able', 'able to', 'about', 'about 15', 'about half', 'about how', 'about it', 'about months', 'about my', 'about that', 'about the', 'about their', 'about them', 'about these', 'about this', 'about what', 'about year', 'about years', 'above', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according', 'according to', 'acid', 'acidic', 'acidity', 'across', 'active', 'actual', 'actually', 'actually tastes', 'add', 'add it', 'add little', 'add some', 'add the', 'add to', 'added', 'added to', 'addict', 'addicted', 'addicted to', 'addicting', 'addictive', 'adding', 'addition', 'addition to', 'additional', 'additionally

In [81]:
print(tfid_counts[3,:].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [82]:
i=0
list_of_sentence=[]
for sentence in final["Text"]:
    list_of_sentence.append(sentence.split())